In [26]:
import gensim
from gensim import corpora
from gensim import similarities
from gensim import models
from gensim.models import CoherenceModel

import preprocess

# Load LDA Model

In [27]:
lda_disk=gensim.models.ldamodel.LdaModel.load("Model/finalmodel_5Topics")

In [28]:
id2word = corpora.Dictionary.load('Model/finalmodel_Dictionary')

# Extract & Preprocess Test Data

In [29]:
test_corpus_1974 = preprocess.load_corpus('Data/Test/Chapters/5827')

test_ids = test_corpus_1974.fileids()
chapters_name = [id.replace('.txt','') for id in test_ids]

test_docs_1974 = preprocess.corpus2docs(test_corpus_1974)

In [31]:
bigram = gensim.models.Phrases(test_docs_1974, min_count=5, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[test_docs_1974], threshold=50)  

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

docs_bigrams = preprocess.make_bigrams(bigram_mod, test_docs_1974)
data_bigrams_trigrams = preprocess.make_trigrams(bigram_mod, trigram_mod, docs_bigrams)

In [32]:
test_vecs_1974 = preprocess.docs2vecs(data_bigrams_trigrams, id2word)


In [33]:
for i in range(0,len(test_vecs_1974)):
    vector = lda_disk[test_vecs_1974[i]]
    sim_topic = max(vector,key=lambda item:item[1])
    print("test doc" + str(i) + ": " + str(vector))
    print("Closest Topic: Topic " + str(sim_topic[0]))

test doc0: [(0, 0.109553985), (1, 0.040380456), (2, 0.7793988), (3, 0.015262106), (4, 0.055404693)]
Closest Topic: Topic 2
test doc1: [(0, 0.03692635), (1, 0.04586395), (2, 0.78906095), (3, 0.016777), (4, 0.11137178)]
Closest Topic: Topic 2
test doc2: [(1, 0.11863371), (2, 0.77049226), (4, 0.09362885)]
Closest Topic: Topic 2
test doc3: [(0, 0.05057396), (1, 0.06623251), (2, 0.6249936), (3, 0.010321293), (4, 0.24787857)]
Closest Topic: Topic 2
test doc4: [(0, 0.012054646), (1, 0.06215301), (2, 0.73367506), (3, 0.022892969), (4, 0.16922428)]
Closest Topic: Topic 2
test doc5: [(0, 0.058278143), (1, 0.0829374), (2, 0.66234493), (3, 0.017452115), (4, 0.17898737)]
Closest Topic: Topic 2
test doc6: [(0, 0.05036419), (1, 0.08894618), (2, 0.6027736), (3, 0.036499497), (4, 0.22141652)]
Closest Topic: Topic 2
test doc7: [(0, 0.027282754), (1, 0.05854047), (2, 0.81275874), (4, 0.09830662)]
Closest Topic: Topic 2
test doc8: [(0, 0.03441285), (1, 0.018538527), (2, 0.5569199), (3, 0.013476093), (4, 0

In [34]:
topic_word = lda_disk.show_topic(0, topn=len(id2word))
topic_word[0:20]

[('conception', 0.032495115),
 ('zarathustra', 0.024165511),
 ('thou', 0.021026777),
 ('hath', 0.015878359),
 ('pure', 0.014599969),
 ('phenomenon', 0.013353135),
 ('thee', 0.013118728),
 ('unto', 0.012600138),
 ('cognition', 0.011412212),
 ('intuition', 0.011389661),
 ('empirical', 0.00927787),
 ('social', 0.009052046),
 ('education', 0.009046041),
 ('activity', 0.009034674),
 ('representation', 0.008575148),
 ('priori', 0.0080437185),
 ('verily', 0.007781609),
 ('doth', 0.00770331),
 ('space', 0.007673117),
 ('transcendental', 0.0074760346)]

## Model 1 - Using Similarity Score

### Top Keywords

In [35]:
chap_num = 0
vector = lda_disk[test_vecs_1974[chap_num]]
sim_topic = max(vector,key=lambda item:item[1])
top_topic = sim_topic[0]
top_topic
topic_word = lda_disk.show_topic(top_topic, topn=len(id2word))
# topic_word

In [36]:
selected_words = [id2word[i[0]] for i in test_vecs_1974[chap_num]]
selected_words[0:10]

['able',
 'accord',
 'acquaint',
 'acquire',
 'action',
 'adopt',
 'advance',
 'afford',
 'agree',
 'almost']

In [37]:
key_words = []

keyword_type = 'bigrams'
for word in topic_word:
    if(keyword_type == 'unigrams'):
        if (len(key_words) < 5) & (word [0] in selected_words):
            key_words.append(word)
    else:
        if ('_' in word[0]) & (len(key_words) < 5) & (word [0] in selected_words):
            key_words.append(word)

key_words

[('precede_chapter', 7.0653455e-06)]

### Recommender

In [46]:
chapters_name

['CHAPTER_III__THE_NATURE_OF_MATTER',
 'CHAPTER_II__THE_EXISTENCE_OF_MATTER',
 'CHAPTER_IV__IDEALISM',
 'CHAPTER_IX__THE_WORLD_OF_UNIVERSALS',
 'CHAPTER_I__APPEARANCE_AND_REALITY',
 'CHAPTER_VIII__HOW__A_PRIORI__KNOWLEDGE_IS_POSSIBLE',
 'CHAPTER_VII__ON_OUR_KNOWLEDGE_OF_GENERAL_PRINCIPLES',
 'CHAPTER_VI__ON_INDUCTION',
 'CHAPTER_V__KNOWLEDGE_BY_ACQUAINTANCE_AND_KNOWLEDGE_BY_DESCRIPTION',
 'CHAPTER_XIII__KNOWLEDGE,_ERROR,_AND_PROBABLE_OPINION',
 'CHAPTER_XII__TRUTH_AND_FALSEHOOD',
 'CHAPTER_XIV__THE_LIMITS_OF_PHILOSOPHICAL_KNOWLEDGE',
 'CHAPTER_XI__ON_INTUITIVE_KNOWLEDGE',
 'CHAPTER_X__ON_OUR_KNOWLEDGE_OF_UNIVERSALS']

In [47]:
chap_num = 0

chosen_chapter = chapters_name[chap_num]
recommendation_scores = []
similarity = similarities.MatrixSimilarity(lda_disk[test_vecs_1974])

for i in range(0,len(test_vecs_1974)):
    vector = lda_disk[test_vecs_1974[i]]
    sim_topic = max(vector,key=lambda item:item[1])
    
    if(i == chap_num):
        sims = similarity[vector]
        sims = list(enumerate(sims))
        for sim in sims:
            chapter_num = sim[0]
            recommendation_score = [chapters_name[chapter_num], sim[1]]
            recommendation_scores.append(recommendation_score)
        
recommendation_scores = sorted(recommendation_scores, key=lambda x: x[1], reverse=True)     
recommendation = []


print('Top 3 Recommended Chapters:')
for i in range(1,4):
    recommendation.append((recommendation_scores[i][0], recommendation_scores[i][1]))
    print(f'{i}. {str(recommendation_scores[i][0])}({str(recommendation_scores[i][1])})')

Top 3 Recommended Chapters:
1. CHAPTER_II__THE_EXISTENCE_OF_MATTER(0.99330914)
2. CHAPTER_VI__ON_INDUCTION(0.9928178)
3. CHAPTER_XIV__THE_LIMITS_OF_PHILOSOPHICAL_KNOWLEDGE(0.99125516)


In [48]:
similarity[lda_disk[test_vecs_1974[0]]]

array([1.        , 0.99330914, 0.983984  , 0.951087  , 0.9799852 ,
       0.97799855, 0.9555314 , 0.99281776, 0.86357546, 0.94942135,
       0.9680863 , 0.99125516, 0.9885328 , 0.9340683 ], dtype=float32)

In [49]:
lda_disk[test_vecs_1974[0]]

[(0, 0.10955375),
 (1, 0.04038038),
 (2, 0.7793999),
 (3, 0.015261371),
 (4, 0.055404633)]

## Model 2 - Using Distribution of Topics

In [50]:
chapters_name

['CHAPTER_III__THE_NATURE_OF_MATTER',
 'CHAPTER_II__THE_EXISTENCE_OF_MATTER',
 'CHAPTER_IV__IDEALISM',
 'CHAPTER_IX__THE_WORLD_OF_UNIVERSALS',
 'CHAPTER_I__APPEARANCE_AND_REALITY',
 'CHAPTER_VIII__HOW__A_PRIORI__KNOWLEDGE_IS_POSSIBLE',
 'CHAPTER_VII__ON_OUR_KNOWLEDGE_OF_GENERAL_PRINCIPLES',
 'CHAPTER_VI__ON_INDUCTION',
 'CHAPTER_V__KNOWLEDGE_BY_ACQUAINTANCE_AND_KNOWLEDGE_BY_DESCRIPTION',
 'CHAPTER_XIII__KNOWLEDGE,_ERROR,_AND_PROBABLE_OPINION',
 'CHAPTER_XII__TRUTH_AND_FALSEHOOD',
 'CHAPTER_XIV__THE_LIMITS_OF_PHILOSOPHICAL_KNOWLEDGE',
 'CHAPTER_XI__ON_INTUITIVE_KNOWLEDGE',
 'CHAPTER_X__ON_OUR_KNOWLEDGE_OF_UNIVERSALS']

In [51]:
# if first digit in tuple matches, store doc
# reco_docs -- dont remove user input
# - get index of user input within reco_docs
# - compare difference between first topic probability and store as list; e.g. {4: 0.002,...}
# - take top 5 minimum difference

j = 0
vector_selected = lda_disk[test_vecs_1974[j]]
vector_selected.sort(key=lambda x: x[1], reverse=True)
vector_selected

reco_docs = {}

for i in range(0,len(test_vecs_1974)):
    vector = lda_disk[test_vecs_1974[i]]
    vector.sort(key=lambda x: x[1], reverse=True)
    
    if (vector[0][0]==vector_selected[0][0]) & (vector[1][0] == vector_selected[1][0]):
        reco_docs[i] = vector
        # print(vector[0][1])

# print(reco_docs)
input_topic = reco_docs[j]

# Remove user input
if j in reco_docs.keys():
    reco_docs.pop(j)

# if there's no 2 same topics
if reco_docs == {}:
    for i in range(0,len(test_vecs_1974)):
        vector = lda_disk[test_vecs_1974[i]]
        vector.sort(key=lambda x: x[1], reverse=True)
        
        if (vector[0][0]==vector_selected[0][0]):
            reco_docs[i] = vector

    # Remove user input
    if j in reco_docs.keys():
        reco_docs.pop(j)

diff = ""
diff_dict = {}
for x, y in reco_docs.items():
    diff = abs(input_topic[0][1] - reco_docs[x][0][1])
    diff_dict[x] = diff

diff_dict = sorted(diff_dict.items(), key=lambda x:x[1], reverse=False)
diff_dict = dict(diff_dict)
# print(diff_dict)

lst = list(diff_dict.keys())
# print(lst[0:5])

print('Top 3 Recommended Chapters:')
for i in range(0,len(lst)):
    if i < 3:
        print(f'{i+1}. {chapters_name[lst[i]]}')

Top 3 Recommended Chapters:
1. CHAPTER_IV__IDEALISM
2. CHAPTER_II__THE_EXISTENCE_OF_MATTER
3. CHAPTER_VI__ON_INDUCTION
